In [3]:
# finetuning bert language model for classification
!pip3 install transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [5]:
# import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# load in pretrained bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

In [ ]:
# load in dataset
df = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Github_Repos/Twitter-Sentiment-Analysis/data/sampled/training.1600000.processed.noemoticon.csv", encoding = "latin-1", low_memory=False)

In [ ]:
# finetune bert model to dataset to classify tweets as positive or negative
# create dataset class
class TweetDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.tweet = dataframe.text
        self.targets = self.data.target
        self.max_len = max_len

    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        tweet = str(self.tweet[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            tweet,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'tweet_text': tweet,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
# create dataloader class
class TweetDataLoader:
    def __init__(self, tweets, tokenizer, max_len, batch_size):
        self.tweets = tweets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.batch_size = batch_size
        self.data_loader = DataLoader(
            self.tweets,
            batch_size=self.batch_size,
            num_workers=4
        )

    def return_data_loader(self):
        return self.data_loader

In [ ]:
# create model class
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
# create training class
class TweetTrainer:
    def __init__(self, model, data_loader, optimizer, device, scheduler=None):
        self.model = model
        self.data_loader = data_loader
        self.optimizer = optimizer
        self.device = device
        self.scheduler = scheduler
        self.loss_fn = nn.CrossEntropyLoss().to(device)

    def train(self):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0

        for d in self.data_loader:
            input_ids = d["input_ids"].to(self.device)
            attention_mask = d["attention_mask"].to(self.device)
            targets = d["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs, dim=1)
            loss = self.loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        return correct_predictions.double() / len(self.data_loader.dataset), np.mean(losses)

In [ ]:
# create testing class
class TweetTester:
    def __init__(self, model, data_loader, device):
        self.model = model
        self.data_loader = data_loader
        self.device = device
        self.loss_fn = nn.CrossEntropyLoss().to(device)

    def test(self):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        with torch.no_grad():
            for d in self.data_loader:
                input_ids = d["input_ids"].to(self.device)
                attention_mask = d["attention_mask"].to(self.device)
                targets = d["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                _, preds = torch.max(outputs, dim=1)
                loss = self.loss_fn(outputs, targets)

                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())

        return correct_predictions.double() / len(self.data_loader.dataset), np.mean(losses)

In [ ]:
# create training function
def train(model, data_loader, optimizer, device, scheduler=None):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

In [ ]:
# create testing function
def test(model, data_loader, device):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

In [ ]:
# create main function
def main():
    # create dataset
    train_dataset = TweetDataset(
        tweets=train_df.text.to_numpy(),
        targets=train_df.target.to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_dataset = TweetDataset(
        tweets=test_df.text.to_numpy(),
        targets=test_df.target.to_numpy(),
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    # create dataloader
    train_data_loader = TweetDataLoader(
        tweets=train_dataset,
        tokenizer=tokenizer,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    ).return_data_loader()

    test_data_loader = TweetDataLoader(
        tweets=test_dataset,
        tokenizer=tokenizer,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    ).return_data_loader()

    # create model
    model = SentimentClassifier(len(class_names))
    model = model.to(device)
    
    # create optimizer
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False)
    total_steps = len(train_data_loader) * EPOCHS
    
    # create scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    # create trainer and tester
    trainer = TweetTrainer(
        model=model,
        data_loader=train_data_loader,
        optimizer=optimizer,
        device=device,
        scheduler=scheduler
    )
    
    tester = TweetTester(
        model=model,
        data_loader=test_data_loader,
        device=device
    )
    
    # create history
    history = defaultdict(list)
    best_accuracy = 0
    
    # start training
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        print("-" * 10)
        
        train_acc, train_loss = trainer.train()
        print(f"Train loss {train_loss} accuracy {train_acc}")
        
        test_acc, test_loss = tester.test()
        print(f"Test loss {test_loss} accuracy {test_acc}")
        print()
        
        history["train_acc"].append(train_acc)
        history["train_loss"].append(train_loss)
        history["test_acc"].append(test_acc)
        history["test_loss"].append(test_loss)
        
        if test_acc > best_accuracy:
            torch.save(model.state_dict(), "best_model_state.bin")
            best_accuracy = test_acc
    
    print(f"Best Test Accuracy: {best_accuracy}")
    
    # plot history
    plt.plot(history["train_acc"], label="train accuracy")
    plt.plot(history["test_acc"], label="test accuracy")
    plt.title("Training history")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend()
    plt.ylim([0, 1])
    plt.show()

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# specify the parameters, then run model
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
MAX_LEN = 160

In [ ]:
# run model
main()